In [66]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import sys
import PIL
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

DIRDATA = "./todaslascaras"
#CATEGORIES = ["anger", "disgust"]
CATEGORIES = ["anger", "disgust", "fear", "happy", "joy", "kiss", "neutral", "sadness", "surprise"]
ANGER = ["anger"]
OBJECTIVEDATA = "./todaslascaras/augmentation"



Primero espejo todas las imagenes de todas las clases

In [116]:
#Con esto leo la imagen y la devuelvo en el color que me indican
def leer_imagen(img_path, gray_scale=False):
    image_src = cv2.imread(img_path)
    if gray_scale:
        image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    else:
        image_rgb = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    return image_rgb

#Este metodo espeja una imagen que viene por parametro y la devuelve
def espejar_imagen(img_path, gris=False):
    img = leer_imagen(img_path, gris)
    img_mirrored = np.fliplr(img)
    return img_mirrored


In [119]:
def guardar_imagen(img, name, where):
    cv2.imwrite(where + '/' + name, img)

def espejar_todo():
    for category in CATEGORIES:
        path = os.path.join(DIRDATA, category)
        for img in os.listdir(path):
            mirrored_img = espejar_imagen(path+'/'+img, gris = True)
            guardar_imagen(mirrored_img, img + '_mirrored.jpg', where = DIRDATA +'/'+category)


In [120]:
espejar_todo()

In [121]:
def detectar_y_guardar_cara(path, savePath, category):
    for img in os.listdir(path):
        image = cv2.imread(path+'/'+img)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.3,
            minNeighbors=3,
            minSize=(30, 30)
        )
        for (x, y, w, h) in faces:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi_color = image[y:y + h, x:x + w]
            cv2.imwrite(savePath + '/' + category +  str(w) + str(h) + '_faces.jpg', roi_color)


In [122]:
rotate_gen = ImageDataGenerator(rotation_range=22)
def rotar_y_guardar(cantidadImagenes, path, category):
    for img in os.listdir(path):
            i = load_img(OBJECTIVEDATA+'/'+category+'/'+img, color_mode="grayscale", target_size=(120,120))
            i = img_to_array(i)
            i = np.expand_dims(i, axis=0)
            PREFIX = category
            imGen = rotate_gen.flow(i, batch_size=1, save_to_dir=OBJECTIVEDATA+'/'+category, save_prefix=PREFIX, save_format='jpg')
            for x in range(cantidadImagenes):
                batch = imGen.next()

In [123]:
def generar_imagenes():
    for category in CATEGORIES:
        path = os.path.join(DIRDATA, category)
        detectar_y_guardar_cara(path, OBJECTIVEDATA+'/'+category, category)



In [124]:
def rotar_caras():
    for category in CATEGORIES:
        path = os.path.join(DIRDATA, category)
        rotar_y_guardar(6, OBJECTIVEDATA+'/'+category, category)

In [125]:
generar_imagenes()

In [126]:
rotar_caras()